
<h2 align=center><font size = 5>Applied DataScience Capstone: Segmenting And Clustering Neighborhoods in Toronto</font></h2>
<h3 align=center><font size = 5>LAB 03 : Week 3 Visualize Data</font></h3>
<h3 align=center><font size = 5></font></h3>


### Objectives

Wiki Page : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
to generate maps to visualize your neighborhoods and how they cluster together. 



#### Initialize Objects and Libraries

In [42]:
# Use Beautiful Soup Object to do Web Scraping of the URL
!pip install beautifulsoup4
!pip install geocoder

from geopy.geocoders import Nominatim # Convert Address to Latitude and Longitude Values
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

#### Read the Wiki Page and do Web Scraping and store the data in a data frame

In [43]:
# Prints all html without 
# print(soup.get_text())

#data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
#for row in rows:
#    cols = row.find_all('td')
#    cols = [ele.text.strip() for ele in cols]
#    data.append([ele for ele in cols if ele]) # Get rid of empty values
#data

# Define Column Names for Dataframe
column_names = ['PostalCode','Borough', 'Neighborhood']
#Instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)
for row in rows:
    tds = row.find_all('td')
    pc =''
    brh=''
    nbd=''
    for n, td in enumerate(tds):
        #print(f"index {n}: {td.text.strip()}")
        if n == 0: pc = td.text.replace('\n', '')  
        if n == 1: brh = td.text.replace('\n', '')  
        if n == 2: nbd = td.text.replace('\n', '')  
    neighbourhoods = neighbourhoods.append({ 'PostalCode' : pc,'Borough' : brh,'Neighborhood' : nbd }, ignore_index=True)

neighbourhoods.drop(
    axis=0,
    index=0,
    inplace=True
)
neighbourhoods.head(20) 


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"




Now read the data to data frame


#### Display the size of the data frame

In [44]:
neighbourhoods.shape

(180, 3)

#### Drop Rows where the Borough is not assigned. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [45]:
indexNames = neighbourhoods[ neighbourhoods['Borough'] == 'Not assigned' ].index
neighbourhoods.drop(indexNames , inplace=True)
neighbourhoods.shape

(103, 3)

In [34]:
neighbourhoods.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [46]:
neighbourhoods.loc[neighbourhoods.PostalCode =='M5A']

,PostalCode,Borough,Neighborhood
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Display Total Rows of the Data Frame

In [47]:
neighbourhoods.shape

(103, 3)

### Enrich the data with Latitude and Longitude

For getting the location details geopy.geocoders package has been used


In [48]:
column_names = ['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude']
#Instantiate the dataframe
geodf = pd.DataFrame(columns=column_names)

for index, row in neighbourhoods.iterrows():
    pccode = row['PostalCode']
    address = 'Toronto, Ontario, {}'.format(pccode)
    geolocator = Nominatim(user_agent="ny_explorer") 
    g = geolocator.geocode(address)
    if g != None : 
        latitude = g.latitude 
        longitude = g.longitude 
        #print(latitude,longitude) 
    else:
        latitude = '' 
        longitude = ''
    geodf = geodf.append({ 'PostalCode' : row['PostalCode'],
                           'Borough' : row['Borough'],
                           'Neighborhood' : row['Neighborhood'],
                           'Latitude' : latitude,
                           'Longitude' : longitude                             
                         }, ignore_index=True)
       
geodf.head()        

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.6449,-79.3818
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",,
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6449,-79.3818


#### Here the library doesnt have coordinates for all the locations

In [49]:
geodf.head()        
    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.6449,-79.3818
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",,
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6449,-79.3818


In [50]:
geodf.shape  

(103, 5)

#### Filter Borough where the latitude and longitude values are available

In [51]:
#geodf_toronto = geodf[geodf['PostalCode'].str.contains("M7A")]
#df[df['year'].isnull()]
#geodf_toronto = geodf[geodf['Latitude'].notnull()]
geodf_toronto = geodf[geodf['Latitude'] != '' ]
geodf_toronto.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.6449,-79.3818
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6449,-79.3818
6,M1B,Scarborough,"Malvern, Rouge",43.6449,-79.3818
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.6449,-79.3818
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.6441,-79.5889


#### Visualize the data using folium

In [52]:
import folium
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer") 
location = geolocator.geocode(address)
latt= location.latitude
long = location.longitude


# Instantiate map of Toronto
map_toronto = folium.Map(location=[latt,long],zoom_start=10)
for lat, lng, borough, neighborhood in zip( geodf_toronto['Latitude'],
                                            geodf_toronto['Longitude'],
                                            geodf_toronto['Borough'],
                                            geodf_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html =True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

